In [2]:
get_ipython().run_line_magic('pylab', 'inline')

Populating the interactive namespace from numpy and matplotlib


/home/jonnyli/anaconda3/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['Button']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [3]:
#from tinc import *
import tinc
import os
import djlib.clex as cl
import matplotlib.pyplot as plt
import numpy as np
import djlib as dj
import json

In [4]:
#Enlarge the jupyter notebook so that plots are easier to see 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
A = tinc.Parameter("A", "weighting")
A.values = np.linspace(.1,2,3)

B = tinc.Parameter("B", "weighting")
B.values = np.linspace(0,2,3)

kT = tinc.Parameter("kT", "weighting")
kT.values = np.linspace(.01,.1,3)

mode = tinc.ParameterString("mode", "fit_mode")
mode.values = ["lasso", "genetic_algorithm"]


graph_buffer = tinc.DiskBufferImage("graph", "out.png", "graph_output")

In [6]:
ps = tinc.ParameterSpace("ps")
ps.register_parameters([A, B, kT, mode])
ps.enable_cache()

Validating json with /home/jonnyli/anaconda3/lib/python3.8/site-packages/tinc/tinc_cache_schema.json


Processes DFT hull & energies data

In [7]:
fcc_data = cl.read_corr_comp_formation("ZrN_FCC_query_data.json")
hcp_data = cl.read_corr_comp_formation("ZrN_HCP_query_data.json")
all_comps = np.concatenate((hcp_data["comp"], fcc_data["comp"]), axis=0)
all_energies = np.concatenate(
    (hcp_data["formation_energy"] / 2, fcc_data["formation_energy"]), axis=0
)
points = np.concatenate((all_comps, np.reshape(all_energies, (-1, 1))), axis=1)

hull = cl.ConvexHull(points)
trash, lower_hull_indices = cl.lower_hull(hull)
hull_vertices = points[lower_hull_indices]
hull_vertices = dj.column_sort(hull_vertices, 0)

In [14]:
import plot_convex_hull_fcc as plotter
def make_graph(A, B, kT, mode):
    ce_fits_dir = os.path.join(os.getcwd())
    project = "ZrN_FCC_1.2.0_demo_"
    json_file = "clex_comp_fe.json"
    if mode == "lasso":
        fit_dir = "Lasso_A_%s_B_%s_kt_%s" % (A, B, kT)
        settings = "lasso_settings.json"
    elif mode == "genetic_algorithm":
        fit_dir = "A_%s_B_%s_kt_%s" % (A, B, kT)
        settings = "genetic_alg_settings.json"
    plotter.plot_convex_hull_fcc(os.path.join(ce_fits_dir,fit_dir))
    #print(os.path.join(ce_fits_dir,fit_dir))
    #os.chdir(os.path.join(ce_fits_dir, fit_dir))
    #print(os.getcwd()) #debug
    #os.system("casm-learn -s %s --select 0" % settings)
    #os.system("casm query -k clex,comp -j -o $s" % json_file)
    '''with open(json_file) as f:
        clex_data = json.load(f)
    ce_fe = np.array([a['clex'] for a in clex_data])
    ce_comp = np.array([a['comp'][0] for a in clex_data])
    '''

    '''
    #Plot DFT hull
    plt.plot(hull_vertices[:, 0], hull_vertices[:, 1], color="k", label="DFT Convex Hull",zorder=1)
    '''
    # Plot DFT energies
    '''plt.scatter(
        np.array(hcp_data["comp"]), np.array(hcp_data["formation_energy"]) / 2, label="HCP",zorder=5, color="b"
    )'''
    '''
    plt.scatter(
        np.array(fcc_data["comp"]), np.array(fcc_data["formation_energy"]), label="FCC",zorder=6, color="orange"
    )

    # Plot CE energies
    plt.scatter(ce_comp, ce_fe, label="Cluster Expansion Prediction",zorder=7, color="orange", marker="x")

    # Plot formatting
    plt.hlines(0, 0, 1, linestyles="dashed",color="k",zorder=0)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    plt.legend(fontsize=15)
    plt.xlabel("Composition (x)", fontsize=20)
    plt.ylabel("Formation Energy per Zr (eV)", fontsize=20)

    fig = plt.gcf()
    fig.set_size_inches(8, 6)
    plt.subplots_adjust(left=0.15,bottom=0.11)

    plt.show()
    image_name = project + fit_dir + '.png'
    #figure(figsize=[18.5,10])
    #close() # Avoid showing as an additional graph in jupyter
    
    os.chdir(ce_fits_dir)
    '''
    #Path to pre-made images, using scripts from vasp_scripts
    image_path = os.path.join(ce_fits_dir, fit_dir, 'convex_hull.png')
    #print(image_path) #debug
    file = open(image_path, 'rb')
    return file.read()


def value_changed(value):
    imagedata = ps.run_process(make_graph)
    graph_buffer.data = imagedata

A.register_callback(value_changed)
B.register_callback(value_changed)
kT.register_callback(value_changed)
mode.register_callback(value_changed)

In [15]:
print(A.values,B.values,kT.values,mode.values)

[0.1  1.05 2.  ] [0. 1. 2.] [0.01  0.055 0.1  ] ['lasso', 'genetic_algorithm']


#currently not using this display approach
from ipywidgets import GridspecLayout
grid = GridspecLayout(3, 2, height='300px')
grid[:, 1] = graph_buffer.interactive_widget()
grid[0, 0] = A.interactive_widget()
grid[1, 0] = B.interactive_widget()
grid[2, 0] = kT.interactive_widget()
grid


In [16]:
from ipywidgets import Button, HBox, VBox

box = VBox([A.interactive_widget(), 
            B.interactive_widget(), 
            kT.interactive_widget(), mode.interactive_widget(),
            graph_buffer.interactive_widget()])
graph_buffer._interactive_widget.width = "60%"
box

In [11]:
mode.value='genetic_algorithm'

Unsupported type for arg: mode - <class 'str'>
